### Import Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
from PIL import Image
import matplotlib.pyplot as plt

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
# Download and import visualkeras library
#!pip install visualkeras
#import visualkeras

### Set seed for reproducibility

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Leaf Dataset

In [4]:
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

### Data Preprocessing

Done in the other notebook

### Data Loader

In [5]:
sub_dataset_dir = '/kaggle/input/leaf-dataset-splitted'
training_dir = os.path.join(sub_dataset_dir, 'training')
validation_dir = os.path.join(sub_dataset_dir, 'validation')
test_dir = os.path.join(sub_dataset_dir, 'testing')

In [6]:
from PIL import Image, ImageFilter, ImageEnhance

def transformation(imageArray):
  image = Image.fromarray(np.reshape(imageArray, (256,256)).astype(np.uint8), 'L') # GrayScale Image
  image = image.convert('1') # Binary Image

  image = image.filter(ImageFilter.MaxFilter(3)) # Erosion
  image = image.filter(ImageFilter.MinFilter(3)) # Dilation

  image = image.filter(ImageFilter.EDGE_ENHANCE) # Edge Enhance
  image = image.filter(ImageFilter.FIND_EDGES) # Edge Detection

  imageArray = np.array(image, dtype=np.bool)

  return np.reshape(imageArray, (256, 256, 1))

In [7]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training, validation, and test sets
# Create an instance of ImageDataGenerator with Data Augmentation
train_data_gen = ImageDataGenerator(preprocessing_function=transformation,
                                    rotation_range=30,
                                    height_shift_range=50,
                                    width_shift_range=50,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='reflect')

valid_data_gen = ImageDataGenerator(preprocessing_function=transformation)

test_data_gen = ImageDataGenerator(preprocessing_function=transformation)


# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               color_mode='grayscale',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               seed=seed)

valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               color_mode='grayscale',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)

test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(256,256),
                                              color_mode='grayscale',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=8,
                                              shuffle=False,
                                              seed=seed)

### Model Metadata

In [8]:
# Model configuration
input_shape = (256, 256, 1)
epochs = 100
batch_size = 64
n_classes = 14
weight_decay = 1e-5
model_name = "CNN"

### CNN Model

* Batch Norm
* Early Stopping
* Batch
* Weight Decay
* Weight Initialization
* Data Augmentation

In [9]:
# Model used for the exercise:
# (Conv + ReLU + MaxPool) x 4 + (Conv + ReLU + GlobalPooling) x 1 + FC x 2; no Dropout
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(input_layer)
    batch_norm1 = tfkl.BatchNormalization()(conv1)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool1)
    batch_norm2 = tfkl.BatchNormalization()(conv2)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool2)
    batch_norm3 = tfkl.BatchNormalization()(conv3)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool3)
    batch_norm4 = tfkl.BatchNormalization()(conv4)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(batch_norm4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer=tf.keras.regularizers.l2(weight_decay)
    )(pool4)
    batch_norm5 = tfkl.BatchNormalization()(conv5)
    globalPool = tfkl.GlobalAveragePooling2D()(batch_norm5)

    flattening_layer = tfkl.Flatten(name='Flatten')(globalPool)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    batch_norm6 = tfkl.BatchNormalization()(classifier_layer)
    output_layer = tfkl.Dense(units=n_classes, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(batch_norm6)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [10]:
# Build model
model = build_model(input_shape)
model.summary()

In [11]:
#tfk.utils.plot_model(model)

In [12]:
#visualkeras.layered_view(model, legend=True, spacing=20, scale_xy=10)

### Training

In [13]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('/kaggle/working/models')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [14]:
# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name=model_name)

# Train the model
history = model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks=[callbacks]
).history

In [15]:
# Save best epoch model
best_model_name = "/kaggle/working/models/" + model_name + "_best_" + str(datetime.now().strftime('%b%d_%H-%M-%S'))
model.save(best_model_name)

In [16]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.ylim(0, 1)
plt.title('Accuracy')
plt.legend(loc='upper right')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.ylim(0, 10)
plt.title('Categorical CrossEntropy Loss')
plt.legend(loc='upper right')
plt.grid(alpha=.3)

plt.show()

### Testing

In [17]:
# Evaluate on test
model_test_metrics = model.evaluate(test_gen, return_dict=True)

print()
print(model_test_metrics)

### Save Model

In [29]:
!zip -r /kaggle/working/cnn.zip ./models/CNN_best_Nov16_13-49-25